# Laboratorio 4: Word Embeddings.


### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Felipe Villena](https://fabianvillena.cl/).
- Profesor Auxiliar: [Gabriel Iturra](https://giturra.cl/)


**Instrucciones:**

- Implementar el método de la Word Context Matrix.
- Entrenar Word2Vec y FastText sobre un pequeño corpus.
- Evaluar los embeddings obtenidos en una tarea de clasificación.
- El laboratorio se realiza en grupos de **máximo** 2 personas. Puede ser invidivual pero no es recomendable.
- La entrega es a través de u-cursos a más tardar el día estipulado arriba. No se aceptan atrasos.
- El formato de entrega es este mismo **Jupyter Notebook**.




## Preguntas 💻  

### Parte A: Word Contex Matrix



En esta parte debe crear una matriz palabra contexto, para esto, complete el siguiente template (para esta parte puede utilizar las librerías ```numpy``` y/o ```scipy```). Hint: revise como utilizar matrices sparse de ```scipy```

```python
class WordContextMatrix:

  def __init__(self, vocab_size, window_size, dataset, tokenizer):
    """
    Utilice el constructor para definir los parametros.
    """

    # se sugiere agregar un una estructura de datos para guardar las
    # palabras del vocab y para guardar el conteo de coocurrencia
    # si lo necesita puede agregar más parametros pero no puede cambiar el resto
    ...
    
  def build_vocab(self):
    """
    Utilice este método para construir el vocabulario
    """
    

    # Le puede ser útil considerar un token unk al vocab
    # para palabras fuera del vocab
    ...
  
  def build_matrix(self):
    """
    Utilice este método para crear la palabra contexto
    """
    ...

  def get_matrix(self):
    """
    Utilice este método para obtener la matriz palabra contexto.
    """

    # se recomienda transformar la matrix a un diccionario de embedding.
    # por ejemplo {palabra1:vec1, palabra2:vec2, ...}
    ...

```

puede modificar los parámetros o métodos si lo considera necesario. Para probar la matrix puede utilizar el siguiente corpus.

```python
corpus = [
  "I like deep learning.",
  "I like NLP.",
  "I enjoy flying."
]
```

Obteniendo una matriz parecia a esta:

***Resultado esperado***:

| counts   | I  | like | enjoy | deep | learning | NLP | flying | . |   
|----------|---:|-----:|------:|-----:|---------:|----:|-------:|--:|
| I        | 0  |  2   |  1    |    0 |  0       |   0 | 0      | 0|            
| like     |  2 |    0 |  0    |    1 |  0       |   1 | 0      | 0 |
| enjoy    |  1 |    0 |  0    |    0 |  0       |   0 | 1      | 0 |
| deep     |  0 |    1 |  0    |    0 |  1       |   0 | 0      | 0 |  
| learning |  0 |    0 |  0    |    1 |  0       |   0 | 0      | 1 |          
| NLP      |  0 |    1 |  0    |    0 |  0       |   0 | 0      | 1 |
| flying   |  0 |    0 |  1    |    0 |  0       |   0 | 0      | 1 |
| .        |  0 |    0 |  0    |    0 |  1       |   1 | 1      | 0 |

``

Verifique si su matrix es igual a esta utilizando el corpus de ejemplo. Ojo que este es sólo un ejemplo, su algoritmo debe **generalizar** a otros ejemplos.

### **Parte B: Word Embeddings**

En la auxiliar 2 aprendieron como entrenar Word2Vec utilizando gensim. El objetivo de esta parte es comparar los embeddings obtenidos con dos modelos diferentes: Word2Vec y [FastText](https://radimrehurek.com/gensim/models/fasttext.html) (utilizen size=200 en FastText) entrenados en el mismo dataset de diálogos de los Simpson.

In [ ]:
import re
import pandas as pd
from time import time
from collections import defaultdict
import string
import multiprocessing
import os
import gensim
import sklearn
from sklearn import linear_model
from collections import Counter
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report

# word2vec
from gensim.models import Word2Vec, KeyedVectors, FastText
from gensim.models.phrases import Phrases, Phraser
from sklearn.model_selection import train_test_split
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Utilizando el dataset adjunto con la tarea:

In [ ]:
data_file = "dialogue-lines-of-the-simpsons.zip"
df = pd.read_csv(data_file)
stopwords = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/english.txt'
).values
stopwords = Counter(stopwords.flatten().tolist())
df = df.dropna().reset_index(drop=True) # Quitar filas vacias

2021-05-05 17:58:59,568 : INFO : NumExpr defaulting to 2 threads.


**Pregunta 1**: Ayudándose de los pasos vistos en la auxiliar, entrene los modelos Word2Vec y FastText sobre el dataset anterior. **(1 punto)** (Hint, le puede servir explorar un poco los datos)

**Respuesta**:

**Pregunta 2**: Encuentre las palabras mas similares a las siguientes: Lisa, Bart, Homer, Marge. Cúal es la diferencia entre ambos resultados? Por qué ocurre esto? Intente comparar ahora Liisa en ambos modelos (doble i). Cuando escogería uno vs el otro? **(0.5 puntos)**

**Respuesta**:

### **Parte C: Aplicar embeddings para clasificar**

Ahora utilizaremos los embeddings que acabamos de calcular para clasificar palabras basadas en su polaridad (positivas o negativas).

Para esto ocuparemos el lexicón AFINN incluido en la tarea, que incluye una lista de palabras y un 1 si su connotación es positiva y un -1 si es negativa.

In [ ]:
AFINN = 'AFINN_full.csv'
df_afinn = pd.read_csv(AFINN, sep='\t', header=None)

Hint: Para w2v son esperables KeyErrors debido a que no todas las palabras del corpus de los simpsons tendrán una representación en AFINN. Pueden utilizar esta función auxiliar para filtrar las filas en el dataframe que no tienen embeddings (como w2v no tiene token UNK se deben ignorar).

In [ ]:
def try_apply(model,word):
    try:
        aux = model[word]
        return True
    except KeyError:
        #logger.error('Word {} not in dictionary'.format(word))
        return False

**Pregunta 1**: Transforme las palabras del corpus de AFINN a la representación en embedding que acabamos de calcular (con ambos modelos).

Su dataframe final debe ser del estilo [embedding, sentimiento], donde los embeddings corresponden a $X$ y el sentimiento asociado con el embedding a $y$ (positivo/negativo, 1/-1).

Para ambos modelos, separar train y test de acuerdo a la siguiente función. **(0.75 puntos)**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)

**Respuesta**:

**Pregunta 2**: Entrenar una regresión logística (vista en auxiliar) y reportar accuracy, precision, recall, f1 y confusion_matrix para ambos modelos. Por qué se obtienen estos resultados? Cómo los mejorarías? **(0.75 puntos)**

**Respuesta**:

### Parte D: Utilizar embeddings pre-entrendados.

**Pregunta 1**: Replicar la parte anterior utilizando embeddings pre-entrenados en un dataset más grande y obtener mejores resultados. Les puede servir [ésta](https://radimrehurek.com/gensim/downloader.html#module-gensim.downloader) documentacion de gensim **(0.25 puntos)**.

**Respuesta**: